In [7]:
import boto3

In [8]:
s3 = boto3.resource('s3',
aws_access_key_id='',
aws_secret_access_key=''
)

In [11]:
try:
    s3.create_bucket(Bucket='cloud-infra-hw3', CreateBucketConfiguration={
    'LocationConstraint': 'us-west-2'})
except Exception as e:
    print (e)

In [12]:
bucket = s3.Bucket('cloud-infra-hw3')

In [13]:
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'YR4B535EZKEXAY3A',
  'HostId': '/IGhTi9ANz/mR/OEWHjraEhDhdb6yB9g4o9uHXQzOEP4OpBWXEhNw5tiDar1T/MoR909Kb+S9NM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/IGhTi9ANz/mR/OEWHjraEhDhdb6yB9g4o9uHXQzOEP4OpBWXEhNw5tiDar1T/MoR909Kb+S9NM=',
   'x-amz-request-id': 'YR4B535EZKEXAY3A',
   'date': 'Thu, 21 Oct 2021 23:51:51 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1}}

In [16]:
body = open('./table/exp1.csv', 'rb')

In [17]:
o = s3.Object('cloud-infra-hw3', 'test').put(Body=body )

In [18]:
s3.Object('cloud-infra-hw3', 'test').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': '5TFS6P5CPXRNRZFY',
  'HostId': 'At2LskFkSe+h2xvyo+XE4y9CdW2jWh8rDaLcGINx/3AeyMBdGDOsEoywd5Rqv6viraHCc9mtnnA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'At2LskFkSe+h2xvyo+XE4y9CdW2jWh8rDaLcGINx/3AeyMBdGDOsEoywd5Rqv6viraHCc9mtnnA=',
   'x-amz-request-id': '5TFS6P5CPXRNRZFY',
   'date': 'Thu, 21 Oct 2021 23:54:41 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [19]:
dyndb = boto3.resource('dynamodb',
region_name='us-west-2',
aws_access_key_id='',
aws_secret_access_key=''
)

In [20]:
try:
    table = dyndb.create_table(
    TableName='DataTable',
    KeySchema=[
    {
    'AttributeName': 'PartitionKey',
    'KeyType': 'HASH'
    },
    {
    'AttributeName': 'RowKey',
    'KeyType': 'RANGE'
    }
    ],
    AttributeDefinitions=[
    {
    'AttributeName': 'PartitionKey',
    'AttributeType': 'S'
    },
    {
    'AttributeName': 'RowKey',
    'AttributeType': 'S'
    },
    ],
    ProvisionedThroughput={
    'ReadCapacityUnits': 5,
    'WriteCapacityUnits': 5
    }
    )
except Exception as e:
    print (e)
    #if there is an exception, the table may already exist. if so...
    table = dyndb.Table("DataTable")

In [21]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [22]:
print(table.item_count)

0


In [23]:
import csv

In [35]:
with open('./table/experiments.csv', 'r', encoding='utf-8-sig') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        if item[0] == 'Id':
            continue
        print(item)
        body = open('./table/'+item[4], 'rb')
        s3.Object('cloud-infra-hw3', item[4]).put(Body=body )
        md = s3.Object('cloud-infra-hw3', item[4]).Acl().put(ACL='public-read')
        url = " https://s3-us-west-2.amazonaws.com/cloud-infra-hw3/"+item[4]
        metadata_item = {'PartitionKey': 'experiment'+item[0], 'RowKey': item[0],
        'Temp' : item[1], 'Conductivity' : item[2], 'Concentration': item[3], 'url':url}
        try:
            table.put_item(Item=metadata_item)
        except:
            print("item may already be there or another failure")

['1', '-1', '52', '3.4', 'exp1.csv']
['2', '-2', '52.1', '3.4', 'exp2.csv']
['3', '-2.93', '57.1', '3.7', 'exp3.csv']


In [36]:
response = table.get_item(
Key={
'PartitionKey': 'experiment3',
'RowKey': '3'
}
)
item = response['Item']
print(item)

{'Temp': '-2.93', 'RowKey': '3', 'Conductivity': '57.1', 'Concentration': '3.7', 'PartitionKey': 'experiment3', 'url': ' https://s3-us-west-2.amazonaws.com/cloud-infra-hw3/exp3.csv'}


In [37]:
response

{'Item': {'Temp': '-2.93',
  'RowKey': '3',
  'Conductivity': '57.1',
  'Concentration': '3.7',
  'PartitionKey': 'experiment3',
  'url': ' https://s3-us-west-2.amazonaws.com/cloud-infra-hw3/exp3.csv'},
 'ResponseMetadata': {'RequestId': 'A2EK8KP73RGH90ICCGP82VDMD7VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 22 Oct 2021 00:31:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '216',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'A2EK8KP73RGH90ICCGP82VDMD7VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3887122851'},
  'RetryAttempts': 0}}